In [1]:
"""
Optuna example that demonstrates a pruner for XGBoost.
In this example, we optimize the validation accuracy of cancer detection using XGBoost.
We optimize both the choice of booster model and their hyperparameters. Throughout
training of models, a pruner observes intermediate results and stop unpromising trials.
You can run this example as follows:
    $ python xgboost_integration.py
"""

import numpy as np
import optuna

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb


# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)

    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
    bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy

In [2]:
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="maximize"
)
study.optimize(objective, n_trials=100)
print(study.best_trial)

[I 2021-08-28 16:25:02,984] A new study created in memory with name: no-name-4a4ccea2-5e46-46f6-9633-7cd2a43112a5


[0]	validation-auc:0.90217
[1]	validation-auc:0.97101
[2]	validation-auc:0.97762
[3]	validation-auc:0.98316
[4]	validation-auc:0.98231
[5]	validation-auc:0.98316
[6]	validation-auc:0.98359
[7]	validation-auc:0.98487
[8]	validation-auc:0.98615
[9]	validation-auc:0.98849


[I 2021-08-28 16:25:03,016] Trial 0 finished with value: 0.9370629370629371 and parameters: {'booster': 'gblinear', 'lambda': 2.104735809177423e-05, 'alpha': 3.3095264565262413e-06}. Best is trial 0 with value: 0.9370629370629371.


[0]	validation-auc:0.88316
[1]	validation-auc:0.88316
[2]	validation-auc:0.88316
[3]	validation-auc:0.88316
[4]	validation-auc:0.88316
[5]	validation-auc:0.88316
[6]	validation-auc:0.88316
[7]	validation-auc:0.88316
[8]	validation-auc:0.88316
[9]	validation-auc:0.88316


[I 2021-08-28 16:25:03,056] Trial 1 finished with value: 0.8741258741258742 and parameters: {'booster': 'dart', 'lambda': 0.2132321619761001, 'alpha': 9.162253607640591e-07, 'max_depth': 1, 'eta': 5.111638849135322e-08, 'gamma': 1.4060002184274652e-05, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 0.00044973343086083774, 'skip_drop': 0.225748931139502}. Best is trial 0 with value: 0.9370629370629371.


...

[0]	validation-auc:0.91112
[1]	validation-auc:0.93532
[2]	validation-auc:0.94320
[3]	validation-auc:0.94533
[4]	validation-auc:0.95311


[I 2021-08-28 16:25:10,753] Trial 98 pruned. Trial was pruned at iteration 5.


[0]	validation-auc:0.84247
[1]	validation-auc:0.93791
[2]	validation-auc:0.96223
[3]	validation-auc:0.96819
[4]	validation-auc:0.98071
[5]	validation-auc:0.97855


[I 2021-08-28 16:25:10,838] Trial 99 pruned. Trial was pruned at iteration 5.


FrozenTrial(number=23, values=[0.993006993006993], datetime_start=datetime.datetime(2021, 8, 28, 16, 25, 4, 468293), datetime_complete=datetime.datetime(2021, 8, 28, 16, 25, 4, 557295), params={'booster': 'dart', 'lambda': 0.0003972573585417277, 'alpha': 0.04940147924246267, 'max_depth': 5, 'eta': 0.6619806282566565, 'gamma': 2.245363206261995e-07, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 0.016321455918765698, 'skip_drop': 1.2417685466798311e-05}, distributions={'booster': CategoricalDistribution(choices=('gbtree', 'gblinear', 'dart')), 'lambda': LogUniformDistribution(high=1.0, low=1e-08), 'alpha': LogUniformDistribution(high=1.0, low=1e-08), 'max_depth': IntUniformDistribution(high=9, low=1, step=1), 'eta': LogUniformDistribution(high=1.0, low=1e-08), 'gamma': LogUniformDistribution(high=1.0, low=1e-08), 'grow_policy': CategoricalDistribution(choices=('depthwise', 'lossguide')), 'sample_type': CategoricalDistribution(choices=('un

In [3]:
study.best_params

{'booster': 'dart',
 'lambda': 0.0003972573585417277,
 'alpha': 0.04940147924246267,
 'max_depth': 5,
 'eta': 0.6619806282566565,
 'gamma': 2.245363206261995e-07,
 'grow_policy': 'depthwise',
 'sample_type': 'weighted',
 'normalize_type': 'forest',
 'rate_drop': 0.016321455918765698,
 'skip_drop': 1.2417685466798311e-05}